In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import roc_auc_score, f1_score
from scipy.stats import entropy
from gensim.models import Word2Vec
import time
import gc
import os
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tqdm.autonotebook import *
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
import tensorflow as tf
import random as rn
import logging
import gensim
pd.set_option('display.max_columns', None)


In [5]:
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df


In [3]:
print('=============================================== read train ===============================================')
t = time.time()
train_df = pd.read_csv('../data/train.csv')
train_df['date'] = pd.to_datetime(
    train_df['ts'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x / 1000)))
)
train_df['day'] = train_df['date'].dt.day
train_df.loc[train_df['day'] == 7, 'day'] = 8
train_df['hour'] = train_df['date'].dt.hour
train_df['minute'] = train_df['date'].dt.minute
train_num = train_df.shape[0]
labels = train_df['target'].values
print('runtime:', time.time() - t)

print('=============================================== click data ===============================================')
click_df = train_df[train_df['target'] == 1].sort_values('timestamp').reset_index(drop=True)
click_df['exposure_click_gap'] = click_df['timestamp'] - click_df['ts']
click_df = click_df[click_df['exposure_click_gap'] >= 0].reset_index(drop=True)
click_df['date'] = pd.to_datetime(
    click_df['timestamp'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x / 1000)))
)
click_df['day'] = click_df['date'].dt.day
click_df.loc[click_df['day'] == 7, 'day'] = 8
del train_df['target'], train_df['timestamp']
for f in ['date', 'exposure_click_gap', 'timestamp', 'ts', 'target', 'hour', 'minute']:
    del click_df[f]
print('runtime:', time.time() - t)

print('=============================================== read test ===============================================')
test_df = pd.read_csv('../data/test.csv')
test_df['date'] = pd.to_datetime(
    test_df['ts'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x / 1000)))
)
test_df['day'] = test_df['date'].dt.day
test_df.loc[test_df['day'] == 10, 'day'] = 11
test_df['hour'] = test_df['date'].dt.hour
test_df['minute'] = test_df['date'].dt.minute
df = pd.concat([train_df, test_df], axis=0, ignore_index=False)
del train_df, test_df, df['date']
gc.collect()
print('runtime:', time.time() - t)

=============================================== read train ===============================================
runtime: 53.88511657714844
=============================================== click data ===============================================
runtime: 60.337608337402344
=============================================== read test ===============================================
runtime: 82.82342743873596


In [4]:
print('=============================================== cate enc ===============================================')
df['lng_lat'] = df['lng'].astype('str') + '_' + df['lat'].astype('str')
del df['guid']
click_df['lng_lat'] = click_df['lng'].astype('str') + '_' + click_df['lat'].astype('str')
sort_df = df.sort_values('ts').reset_index(drop=True)
cate_cols = [
    'deviceid', 'newsid', 'pos', 'app_version', 'device_vendor',
    'netmodel', 'osversion', 'device_version', 'lng', 'lat', 'lng_lat'
]
for f in cate_cols:
    print(f)
    if(f == 'deviceid'):
        map_dict_deviceid = dict(zip(df[f].unique(), range(df[f].nunique())))
        df[f] = df[f].map(map_dict_deviceid).fillna(-1).astype('int32')
        click_df[f] = click_df[f].map(map_dict_deviceid).fillna(-1).astype('int32')
        sort_df[f] = sort_df[f].map(map_dict_deviceid).fillna(-1).astype('int32')
        df[f + '_count'] = df[f].map(df[f].value_counts())
    if(f == 'newsid'):
        map_dict_newsid = dict(zip(df[f].unique(), range(df[f].nunique())))
        df[f] = df[f].map(map_dict_newsid).fillna(-1).astype('int32')
        click_df[f] = click_df[f].map(map_dict_newsid).fillna(-1).astype('int32')
        sort_df[f] = sort_df[f].map(map_dict_newsid).fillna(-1).astype('int32')
        df[f + '_count'] = df[f].map(df[f].value_counts())
    if(f == 'lng_lat'):
        map_dict_ll = dict(zip(df[f].unique(), range(df[f].nunique())))
        df[f] = df[f].map(map_dict_ll).fillna(-1).astype('int32')
        click_df[f] = click_df[f].map(map_dict_ll).fillna(-1).astype('int32')
        sort_df[f] = sort_df[f].map(map_dict_ll).fillna(-1).astype('int32')
        df[f + '_count'] = df[f].map(df[f].value_counts())
    
    map_dict = dict(zip(df[f].unique(), range(df[f].nunique())))
    df[f] = df[f].map(map_dict).fillna(-1).astype('int32')
    click_df[f] = click_df[f].map(map_dict).fillna(-1).astype('int32')
    sort_df[f] = sort_df[f].map(map_dict).fillna(-1).astype('int32')
    df[f + '_count'] = df[f].map(df[f].value_counts())
df = reduce_mem(df)
click_df = reduce_mem(click_df)
sort_df = reduce_mem(sort_df)
del map_dict
gc.collect()
print('runtime:', time.time() - t)

=============================================== cate enc ===============================================
deviceid
newsid
pos
app_version
device_vendor
netmodel
osversion
device_version
lng
lat
lng_lat
2580.12 Mb, 1390.40 Mb (46.11 %)
77.91 Mb, 46.97 Mb (39.71 %)
1204.05 Mb, 673.70 Mb (44.05 %)
runtime: 237.66832995414734


In [5]:
print('=============================================== feat eng ===============================================')

print('*************************** history stats ***************************')
for f in [
    ['deviceid'],
    ['pos', 'deviceid'],
    ['newsid'],
    ['newsid','pos'],
    # ...
]:
    print('------------------ {} ------------------'.format('_'.join(f)))
    
    # 对前一天的点击次数进行统计
    tmp = click_df[f + ['day', 'id']].groupby(f + ['day'], as_index=False)['id'].agg({'_'.join(f) + '_prev_day_click_count': 'count'})
    tmp['day'] += 1
    df = df.merge(tmp, on=f + ['day'], how='left')
    df['_'.join(f) + '_prev_day_click_count'] = df['_'.join(f) + '_prev_day_click_count'].fillna(0)
    df.loc[df['day'] == 8, '_'.join(f) + '_prev_day_click_count'] = None
    
    # 对前一天的曝光量进行统计
    tmp = df[f + ['day', 'id']].groupby(f + ['day'], as_index=False)['id'].agg({'_'.join(f) + '_prev_day_count': 'count'})
    tmp['day'] += 1
    df = df.merge(tmp, on=f + ['day'], how='left')
    df['_'.join(f) + '_prev_day_count'] = df['_'.join(f) + '_prev_day_count'].fillna(0)
    df.loc[df['day'] == 8, '_'.join(f) + '_prev_day_count'] = None
    
    # 计算前一天的点击率
    df['_'.join(f) + '_prev_day_ctr'] = df['_'.join(f) + '_prev_day_click_count'] / (
            df['_'.join(f) + '_prev_day_count'] + df['_'.join(f) + '_prev_day_count'].mean())

    del tmp
    print('runtime:', time.time() - t)
del click_df
df = reduce_mem(df)

print('*************************** exposure_ts_gap ***************************')
for f in [
    ['deviceid'], ['newsid'], ['lng_lat'],
    ['pos', 'deviceid'], ['pos', 'newsid'], ['pos', 'lng_lat'],
    ['pos', 'deviceid', 'lng_lat'],
    ['netmodel', 'deviceid'],
    ['pos', 'netmodel', 'deviceid'],
    ['netmodel', 'lng_lat'], ['deviceid', 'lng_lat'],
    ['netmodel', 'deviceid', 'lng_lat'], ['pos', 'netmodel', 'lng_lat'],
    ['pos', 'netmodel', 'deviceid', 'lng_lat']
]:
    print('------------------ {} ------------------'.format('_'.join(f)))

    tmp = sort_df[f + ['ts']].groupby(f)
    # 前x次、后x次曝光到当前的时间差
    for gap in [1, 2, 3, 5, 10]:
        sort_df['{}_prev{}_exposure_ts_gap'.format('_'.join(f), gap)] = tmp['ts'].shift(0) - tmp['ts'].shift(gap)
        sort_df['{}_next{}_exposure_ts_gap'.format('_'.join(f), gap)] = tmp['ts'].shift(-gap) - tmp['ts'].shift(0)
        tmp2 = sort_df[
            f + ['ts', '{}_prev{}_exposure_ts_gap'.format('_'.join(f), gap), '{}_next{}_exposure_ts_gap'.format('_'.join(f), gap)]
        ].drop_duplicates(f + ['ts']).reset_index(drop=True)
        df = df.merge(tmp2, on=f + ['ts'], how='left')
        del sort_df['{}_prev{}_exposure_ts_gap'.format('_'.join(f), gap)]
        del sort_df['{}_next{}_exposure_ts_gap'.format('_'.join(f), gap)]
        del tmp2

    del tmp
    df = reduce_mem(df)
    print('runtime:', time.time() - t)
del df['ts']
gc.collect()

print('*************************** cross feat (second order) ***************************')
# 二阶交叉特征，可以继续做更高阶的交叉。
cross_cols = ['deviceid', 'newsid', 'pos', 'netmodel', 'lng_lat']
for f in cross_cols:
    for col in cross_cols:
        if col == f:
            continue
        print('------------------ {} {} ------------------'.format(f, col))
        df = df.merge(df[[f, col]].groupby(f, as_index=False)[col].agg({
            'cross_{}_{}_nunique'.format(f, col): 'nunique',
            'cross_{}_{}_ent'.format(f, col): lambda x: entropy(x.value_counts() / x.shape[0]) # 熵
        }), on=f, how='left')
        if 'cross_{}_{}_count'.format(f, col) not in df.columns.values and 'cross_{}_{}_count'.format(col, f) not in df.columns.values:
            df = df.merge(df[[f, col, 'id']].groupby([f, col], as_index=False)['id'].agg({
                'cross_{}_{}_count'.format(f, col): 'count' # 共现次数
            }), on=[f, col], how='left')
        if 'cross_{}_{}_count_ratio'.format(col, f) not in df.columns.values:
            df['cross_{}_{}_count_ratio'.format(col, f)] = df['cross_{}_{}_count'.format(f, col)] / df[f + '_count'] # 比例偏好
        if 'cross_{}_{}_count_ratio'.format(f, col) not in df.columns.values:
            df['cross_{}_{}_count_ratio'.format(f, col)] = df['cross_{}_{}_count'.format(f, col)] / df[col + '_count'] # 比例偏好
        df['cross_{}_{}_nunique_ratio_{}_count'.format(f, col, f)] = df['cross_{}_{}_nunique'.format(f, col)] / df[f + '_count']
        print('runtime:', time.time() - t)
    df = reduce_mem(df)
del df['id']
gc.collect()



=============================================== feat eng ===============================================
*************************** history stats ***************************
------------------ deviceid ------------------
runtime: 258.8171241283417
------------------ pos_deviceid ------------------
runtime: 284.7622137069702
------------------ newsid ------------------
runtime: 315.93544578552246
------------------ newsid_pos ------------------
runtime: 355.2394235134125
2766.46 Mb, 1734.41 Mb (37.31 %)
*************************** exposure_ts_gap ***************************
------------------ deviceid ------------------
2881.13 Mb, 2307.77 Mb (19.90 %)
runtime: 496.04815316200256
------------------ newsid ------------------
3454.49 Mb, 2881.13 Mb (16.60 %)
runtime: 632.4443359375
------------------ lng_lat ------------------
4027.85 Mb, 3454.49 Mb (14.23 %)
runtime: 765.5021119117737
------------------ pos_deviceid ------------------
4601.21 Mb, 4027.85 Mb (12.46 %)
runtime: 916.694189

0

In [8]:
def emb(df, f1, f2):
    emb_size = 8
    print('====================================== {} {} ======================================'.format(f1, f2))
    tmp = df.groupby(f1, as_index=False)[f2].agg({'{}_{}_list'.format(f1, f2): list})
    sentences = tmp['{}_{}_list'.format(f1, f2)].values.tolist()
    del tmp['{}_{}_list'.format(f1, f2)]
    for i in range(len(sentences)):
        sentences[i] = [str(x) for x in sentences[i]]
    model = Word2Vec(sentences, size=emb_size, window=5, min_count=5, sg=0, hs=1, seed=2019)
    emb_matrix = []
    for seq in sentences:
        vec = []
        for w in seq:
            if w in model:
                 vec.append(model[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * emb_size)
    emb_matrix = np.array(emb_matrix)
    for i in range(emb_size):
        tmp['{}_{}_emb_{}'.format(f1, f2, i)] = emb_matrix[:, i]
    del model, emb_matrix, sentences
    tmp = reduce_mem(tmp)
    print('runtime:', time.time() - t)
    return tmp


emb_cols = [
    ['deviceid', 'newsid'],
    ['deviceid', 'lng_lat'],
    ['newsid', 'lng_lat'],
    # ...
]
for f1, f2 in emb_cols:
    df = df.merge(emb(sort_df, f1, f2), on=f1, how='left')
    df = df.merge(emb(sort_df, f2, f1), on=f2, how='left')
del sort_df
gc.collect()


====================================== deviceid newsid ======================================


/home/zxh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/home/zxh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


8.74 Mb, 3.06 Mb (65.00 %)
runtime: 13617.837913513184
====================================== newsid deviceid ======================================
100.68 Mb, 35.24 Mb (65.00 %)
runtime: 14052.586945056915
====================================== deviceid lng_lat ======================================
8.74 Mb, 3.06 Mb (65.00 %)
runtime: 14283.030943393707
====================================== lng_lat deviceid ======================================
35.64 Mb, 12.48 Mb (65.00 %)
runtime: 14532.700240135193
====================================== newsid lng_lat ======================================
100.68 Mb, 35.24 Mb (65.00 %)
runtime: 14984.698087215424
====================================== lng_lat newsid ======================================
35.64 Mb, 12.48 Mb (65.00 %)
runtime: 15316.422714471817


0

In [6]:
print('========================================================================================================')
train_df = df[:train_num].reset_index(drop=True)
train_df = reduce_mem(train_df)
test_df = df[train_num:].reset_index(drop=True)
test_df = reduce_mem(test_df)
del df
gc.collect()

7215.02 Mb, 7215.02 Mb (0.00 %)
2317.08 Mb, 2317.08 Mb (0.00 %)


0

In [2]:
print('========================================================================================================')
train_df = df[:train_num].reset_index(drop=True)
test_df = df[train_num:].reset_index(drop=True)
del df
gc.collect()

train_idx = train_df[train_df['day'] < 10].index.tolist()
val_idx = train_df[train_df['day'] == 10].index.tolist()

train_x = train_df.iloc[train_idx].reset_index(drop=True)
train_y = labels[train_idx]
val_x = train_df.iloc[val_idx].reset_index(drop=True)
val_y = labels[val_idx]

# del train_x['day'], val_x['day'], train_df['day'], test_df['day']
gc.collect()
print('runtime:', time.time() - t)
print('========================================================================================================')

                

In [ ]:
train_x.shape,val_x.shape,test_df.shape,train_df.shape

In [7]:
print('*************************** app tag prone  ***************************')
print('====================== app  tag begin =================')

app_prone = pd.read_pickle('../pickle/Graph_prone_app.pickle')
app_prone = reduce_mem(app_prone)
tag_prone = pd.read_pickle('../pickle/Graph_prone_tag.pickle')
tag_prone = reduce_mem(tag_prone)

app_prone['deviceid'] = app_prone['deviceid'].map(map_dict_deviceid).fillna(-1).astype('int32')
tag_prone['deviceid'] = tag_prone['deviceid'].map(map_dict_deviceid).fillna(-1).astype('int32')

#train_x = train_x.merge(app_prone,on = 'deviceid',how = 'left')
#print('~~~~~~~~train_x~~~~~~~~~~~~~~')
#val_x = val_x.merge(app_prone,on = 'deviceid',how = 'left')
#print('~~~~~~~~~val_x~~~~~~~~~~~~~')
test_df = test_df.merge(app_prone,on = 'deviceid',how = 'left')
print('~~~~~~~~~~test_df~~~~~~~~~~~~')
train_df = train_df.merge(app_prone,on = 'deviceid',how = 'left')
print('~~~~~~~~~~train_df~~~~~~~~~~~~')
del app_prone
gc.collect()

#train_x = train_x.merge(tag_prone,on = 'deviceid',how = 'left')
#print('~~~~~~~~train_x~~~~~~~~~~~~~~')
#val_x = val_x.merge(tag_prone,on = 'deviceid',how = 'left')
#print('~~~~~~~~~~val_x~~~~~~~~~~~~')
test_df = test_df.merge(tag_prone,on = 'deviceid',how = 'left')
print('~~~~~~~~~~test_df~~~~~~~~~~~~')
train_df = train_df.merge(tag_prone,on = 'deviceid',how = 'left')
print('~~~~~~~~~~~train_df~~~~~~~~~~~')
print('====================== app  tag end =================')

del tag_prone
gc.collect()

*************************** app tag prone  ***************************
====================== app  tag begin =================
12.24 Mb, 4.37 Mb (64.29 %)
12.24 Mb, 4.37 Mb (64.29 %)
~~~~~~~~~~test_df~~~~~~~~~~~~
~~~~~~~~~~train_df~~~~~~~~~~~~
~~~~~~~~~~test_df~~~~~~~~~~~~
~~~~~~~~~~~train_df~~~~~~~~~~~
====================== app  tag end =================


0

In [8]:
test_df.shape,train_df.shape

((3653592, 202), (11376681, 202))

In [9]:
print('*************************** embedding ***************************')
print('====================== read devid  newsid  prone emb =================')
dev_newsid = pd.read_pickle('../pickle/Graph_prone_all_data_dev_news.pickle')
dev_newsid = reduce_mem(dev_newsid)
newsid_dev = pd.read_pickle('../pickle/Graph_prone_all_data_news_dev.pickle')
newsid_dev = reduce_mem(newsid_dev)
dev_newsid['deviceid'] = dev_newsid['deviceid'].map(map_dict_deviceid).fillna(-1).astype('int32')
newsid_dev['newsid'] = newsid_dev['newsid'].map(map_dict_newsid).fillna(-1).astype('int32')

# train_x = train_x.merge(dev_newsid,on = 'deviceid',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(dev_newsid,on = 'deviceid',how = 'left')
# print('~~~~~~~~val_x~~~~~~~~~~~~~~')
test_df = test_df.merge(dev_newsid,on = 'deviceid',how = 'left')
print('~~~~~~~~test_df~~~~~~~~~~~~~~')
train_df = train_df.merge(dev_newsid,on = 'deviceid',how = 'left')
print('~~~~~~~~train_df~~~~~~~~~~~~~~')


# train_x = train_x.merge(newsid_dev,on = 'newsid',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(newsid_dev,on = 'newsid',how = 'left')
# print('~~~~~~~~val_x~~~~~~~~~~~~~~')
test_df = test_df.merge(newsid_dev,on = 'newsid',how = 'left')
print('~~~~~~~~test_df~~~~~~~~~~~~~~')
train_df = train_df.merge(newsid_dev,on = 'newsid',how = 'left')
print('~~~~~~~~train_df~~~~~~~~~~~~~~')
del dev_newsid,newsid_dev
gc.collect()


*************************** embedding ***************************
====================== read devid  newsid  prone emb =================
12.24 Mb, 4.37 Mb (64.29 %)
140.95 Mb, 50.34 Mb (64.29 %)
~~~~~~~~test_df~~~~~~~~~~~~~~
~~~~~~~~train_df~~~~~~~~~~~~~~
~~~~~~~~test_df~~~~~~~~~~~~~~
~~~~~~~~train_df~~~~~~~~~~~~~~


0

In [10]:
train_df.shape,test_df.shape
# ,train_x.shape

((11376681, 226), (3653592, 226))

In [11]:
print('*************************** embedding ***************************')
print('====================== read devid  lhg_lat  prone emb =================')
dev_ll = pd.read_pickle('../pickle/Graph_prone_all_data_dev_ll.pickle')
dev_ll = reduce_mem(dev_ll)
ll_dev = pd.read_pickle('../pickle/Graph_prone_all_data_ll_dev.pickle')
ll_dev = reduce_mem(ll_dev)
dev_ll['deviceid'] = dev_ll['deviceid'].map(map_dict_deviceid).fillna(-1).astype('int32')
ll_dev['lng_lat'] = ll_dev['lng_lat'].map(map_dict_ll).fillna(-1).astype('int32')

# train_x = train_x.merge(dev_ll,on = 'deviceid',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(dev_ll,on = 'deviceid',how = 'left')
# print('~~~~~~~~val_x~~~~~~~~~~~~~~')
test_df = test_df.merge(dev_ll,on = 'deviceid',how = 'left')
print('~~~~~~~~test_df~~~~~~~~~~~~~~')
train_df = train_df.merge(dev_ll,on = 'deviceid',how = 'left')
print('~~~~~~~~train_df~~~~~~~~~~~~~~')


# train_x = train_x.merge(ll_dev,on = 'lng_lat',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(ll_dev,on = 'lng_lat',how = 'left')
# print('~~~~~~~~val_x~~~~~~~~~~~~~~')
test_df = test_df.merge(ll_dev,on = 'lng_lat',how = 'left')
print('~~~~~~~~test_df~~~~~~~~~~~~~~')
train_df = train_df.merge(ll_dev,on = 'lng_lat',how = 'left')
print('~~~~~~~~train_df~~~~~~~~~~~~~~')
del dev_ll,ll_dev
gc.collect()

*************************** embedding ***************************
====================== read devid  lhg_lat  prone emb =================
12.24 Mb, 4.37 Mb (64.29 %)
49.90 Mb, 17.82 Mb (64.29 %)
~~~~~~~~test_df~~~~~~~~~~~~~~
~~~~~~~~train_df~~~~~~~~~~~~~~
~~~~~~~~test_df~~~~~~~~~~~~~~
~~~~~~~~train_df~~~~~~~~~~~~~~


0

In [12]:
train_df.shape,test_df.shape
# ,train_x.shape

((11376681, 250), (3653592, 250))

In [13]:
print('*************************** embedding ***************************')
print('====================== read newsid  lhg_lat  prone emb =================')
news_ll = pd.read_pickle('../pickle/Graph_prone_all_data_news_ll.pickle')
news_ll = reduce_mem(news_ll)

ll_news = pd.read_pickle('../pickle/Graph_prone_all_data_ll_news.pickle')
ll_news = reduce_mem(ll_news)

news_ll['newsid'] = news_ll['newsid'].map(map_dict_newsid).fillna(-1).astype('int32')
ll_news['lng_lat'] = ll_news['lng_lat'].map(map_dict_ll).fillna(-1).astype('int32')

# train_x = train_x.merge(news_ll,on = 'newsid',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(news_ll,on = 'newsid',how = 'left')
# print('~~~~~~~~val_x~~~~~~~~~~~~~~')
test_df = test_df.merge(news_ll,on = 'newsid',how = 'left')
print('~~~~~~~~test_df~~~~~~~~~~~~~~')
train_df = train_df.merge(news_ll,on = 'newsid',how = 'left')
print('~~~~~~~~train_df~~~~~~~~~~~~~~')


# train_x = train_x.merge(ll_news,on = 'lng_lat',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(ll_news,on = 'lng_lat',how = 'left')
# print('~~~~~~~~val_x~~~~~~~~~~~~~~')
test_df = test_df.merge(ll_news,on = 'lng_lat',how = 'left')
print('~~~~~~~~test_df~~~~~~~~~~~~~~')
train_df = train_df.merge(ll_news,on = 'lng_lat',how = 'left')
print('~~~~~~~~train_df~~~~~~~~~~~~~~')
del news_ll,ll_news
gc.collect()

*************************** embedding ***************************
====================== read newsid  lhg_lat  prone emb =================
140.95 Mb, 50.34 Mb (64.29 %)
49.90 Mb, 17.82 Mb (64.29 %)
~~~~~~~~test_df~~~~~~~~~~~~~~
~~~~~~~~train_df~~~~~~~~~~~~~~
~~~~~~~~test_df~~~~~~~~~~~~~~
~~~~~~~~train_df~~~~~~~~~~~~~~


0

In [14]:
train_df.shape,test_df.shape
# ,train_x.shape

((11376681, 274), (3653592, 274))

In [16]:
print('*************************** prone edges ***************************')
print('====================== app_flatten  =================')

app_flatten = pd.read_csv('../pickle/app_flatten_all_data.csv')

app_flatten['deviceid'] = app_flatten['deviceid'].map(map_dict_deviceid).fillna(-1).astype('int32')


# train_x = train_x.merge(app_flatten,on = 'deviceid',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(app_flatten,on = 'deviceid',how = 'left')
# print('~~~~~~~~~val_x~~~~~~~~~~~~~')
test_df = test_df.merge(app_flatten,on = 'deviceid',how = 'left')
print('~~~~~~~~~~test_df~~~~~~~~~~~~')
train_df = train_df.merge(app_flatten,on = 'deviceid',how = 'left')
print('~~~~~~~~~~train_df~~~~~~~~~~~~')
del app_flatten
gc.collect()


*************************** prone edges ***************************
====================== app_flatten  =================
~~~~~~~~~~test_df~~~~~~~~~~~~
~~~~~~~~~~train_df~~~~~~~~~~~~


0

In [17]:
train_df.shape,test_df.shape
# ,train_x.shape

((11376681, 290), (3653592, 290))

In [18]:
print('*************************** prone edges ***************************')
print('====================== tag  =================')

app_doc2vec = pd.read_csv('../pickle/device_app_doc2vec_emb.csv')
tag_doc2vec = pd.read_csv('../pickle/tag_data_doc2vec_emb.csv')

del app_doc2vec['Unnamed: 0']
del tag_doc2vec['Unnamed: 0']

app_doc2vec['deviceid'] = app_doc2vec['deviceid'].map(map_dict_deviceid).fillna(-1).astype('int32')
tag_doc2vec['deviceid'] = tag_doc2vec['deviceid'].map(map_dict_deviceid).fillna(-1).astype('int32')

# train_x = train_x.merge(app_doc2vec,on = 'deviceid',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(app_doc2vec,on = 'deviceid',how = 'left')
# print('~~~~~~~~~val_x~~~~~~~~~~~~~')
test_df = test_df.merge(app_doc2vec,on = 'deviceid',how = 'left')
print('~~~~~~~~~~test_df~~~~~~~~~~~~')
train_df = train_df.merge(app_doc2vec,on = 'deviceid',how = 'left')
print('~~~~~~~~~~train_df~~~~~~~~~~~~')
del app_doc2vec
gc.collect()

# train_x = train_x.merge(tag_doc2vec,on = 'deviceid',how = 'left')
# print('~~~~~~~~train_x~~~~~~~~~~~~~~')
# val_x = val_x.merge(tag_doc2vec,on = 'deviceid',how = 'left')
# print('~~~~~~~~~~val_x~~~~~~~~~~~~')
test_df = test_df.merge(tag_doc2vec,on = 'deviceid',how = 'left')
print('~~~~~~~~~~test_df~~~~~~~~~~~~')
train_df = train_df.merge(tag_doc2vec,on = 'deviceid',how = 'left')
print('~~~~~~~~~~~train_df~~~~~~~~~~~')
del tag_doc2vec
gc.collect()

*************************** prone edges ***************************
====================== tag  =================
~~~~~~~~~~test_df~~~~~~~~~~~~
~~~~~~~~~~train_df~~~~~~~~~~~~
~~~~~~~~~~test_df~~~~~~~~~~~~
~~~~~~~~~~~train_df~~~~~~~~~~~


0

In [19]:
train_df.shape,test_df.shape
# ,train_x.shape

((11376681, 418), (3653592, 418))

In [20]:
train_df.head()

,id,deviceid,newsid,pos,app_version,device_vendor,netmodel,osversion,lng,lat,device_version,day,hour,minute,lng_lat,deviceid_count,newsid_count,pos_count,app_version_count,device_vendor_count,netmodel_count,osversion_count,device_version_count,lng_count,lat_count,lng_lat_count,deviceid_prev_day_click_count,deviceid_prev_day_count,deviceid_prev_day_ctr,pos_deviceid_prev_day_click_count,pos_deviceid_prev_day_count,pos_deviceid_prev_day_ctr,newsid_prev_day_click_count,newsid_prev_day_count,newsid_prev_day_ctr,newsid_pos_prev_day_click_count,newsid_pos_prev_day_count,newsid_pos_prev_day_ctr,deviceid_prev1_exposure_ts_gap,deviceid_next1_exposure_ts_gap,deviceid_prev2_exposure_ts_gap,deviceid_next2_exposure_ts_gap,deviceid_prev3_exposure_ts_gap,deviceid_next3_exposure_ts_gap,deviceid_prev5_exposure_ts_gap,deviceid_next5_exposure_ts_gap,deviceid_prev10_exposure_ts_gap,deviceid_next10_exposure_ts_gap,newsid_prev1_exposure_ts_gap,newsid_next1_exposure_ts_gap,newsid_prev2_exposure_ts_gap,newsid_next2_exposure_ts_gap,newsid_prev3_exposure_ts_gap,newsid_next3_exposure_ts_gap,newsid_prev5_exposure_ts_gap,newsid_next5_exposure_ts_gap,newsid_prev10_exposure_ts_gap,newsid_next10_exposure_ts_gap,lng_lat_prev1_exposure_ts_gap,lng_lat_next1_exposure_ts_gap,lng_lat_prev2_exposure_ts_gap,lng_lat_next2_exposure_ts_gap,lng_lat_prev3_exposure_ts_gap,lng_lat_next3_exposure_ts_gap,lng_lat_prev5_exposure_ts_gap,lng_lat_next5_exposure_ts_gap,lng_lat_prev10_exposure_ts_gap,lng_lat_next10_exposure_ts_gap,pos_deviceid_prev1_exposure_ts_gap,pos_deviceid_next1_exposure_ts_gap,pos_deviceid_prev2_exposure_ts_gap,pos_deviceid_next2_exposure_ts_gap,pos_deviceid_prev3_exposure_ts_gap,pos_deviceid_next3_exposure_ts_gap,pos_deviceid_prev5_exposure_ts_gap,pos_deviceid_next5_exposure_ts_gap,pos_deviceid_prev10_exposure_ts_gap,pos_deviceid_next10_exposure_ts_gap,pos_newsid_prev1_exposure_ts_gap,pos_newsid_next1_exposure_ts_gap,pos_newsid_prev2_exposure_ts_gap,pos_newsid_next2_exposure_ts_gap,pos_newsid_prev3_exposure_ts_gap,pos_newsid_next3_exposure_ts_gap,pos_newsid_prev5_exposure_ts_gap,pos_newsid_next5_exposure_ts_gap,pos_newsid_prev10_exposure_ts_gap,pos_newsid_next10_exposure_ts_gap,pos_lng_lat_prev1_exposure_ts_gap,pos_lng_lat_next1_exposure_ts_gap,pos_lng_lat_prev2_exposure_ts_gap,pos_lng_lat_next2_exposure_ts_gap,pos_lng_lat_prev3_exposure_ts_gap,pos_lng_lat_next3_exposure_ts_gap,pos_lng_lat_prev5_exposure_ts_gap,pos_lng_lat_next5_exposure_ts_gap,pos_lng_lat_prev10_exposure_ts_gap,pos_lng_lat_next10_exposure_ts_gap,pos_deviceid_lng_lat_prev1_exposure_ts_gap,pos_deviceid_lng_lat_next1_exposure_ts_gap,pos_deviceid_lng_lat_prev2_exposure_ts_gap,pos_deviceid_lng_lat_next2_exposure_ts_gap,pos_deviceid_lng_lat_prev3_exposure_ts_gap,pos_deviceid_lng_lat_next3_exposure_ts_gap,pos_deviceid_lng_lat_prev5_exposure_ts_gap,pos_deviceid_lng_lat_next5_exposure_ts_gap,pos_deviceid_lng_lat_prev10_exposure_ts_gap,pos_deviceid_lng_lat_next10_exposure_ts_gap,netmodel_deviceid_prev1_exposure_ts_gap,netmodel_deviceid_next1_exposure_ts_gap,netmodel_deviceid_prev2_exposure_ts_gap,netmodel_deviceid_next2_exposure_ts_gap,netmodel_deviceid_prev3_exposure_ts_gap,netmodel_deviceid_next3_exposure_ts_gap,netmodel_deviceid_prev5_exposure_ts_gap,netmodel_deviceid_next5_exposure_ts_gap,netmodel_deviceid_prev10_exposure_ts_gap,netmodel_deviceid_next10_exposure_ts_gap,pos_netmodel_deviceid_prev1_exposure_ts_gap,pos_netmodel_deviceid_next1_exposure_ts_gap,pos_netmodel_deviceid_prev2_exposure_ts_gap,pos_netmodel_deviceid_next2_exposure_ts_gap,pos_netmodel_deviceid_prev3_exposure_ts_gap,pos_netmodel_deviceid_next3_exposure_ts_gap,pos_netmodel_deviceid_prev5_exposure_ts_gap,pos_netmodel_deviceid_next5_exposure_ts_gap,pos_netmodel_deviceid_prev10_exposure_ts_gap,pos_netmodel_deviceid_next10_exposure_ts_gap,netmodel_lng_lat_prev1_exposure_ts_gap,netmodel_lng_lat_next1_exposure_ts_gap,netmodel_lng_lat_prev2_exposure_ts_gap,netmodel_lng_lat_next2_exposure_ts_gap,netmodel_lng_lat_prev3_exposure_ts_gap,net

In [15]:
train_df.shape,test_df.shape

((11376681, 274), (3653592, 274))

In [33]:
# average score: 0.9840631756268536 线上 0.81467
feas2drop = [
    #'lng', 'lat', 'lng_lat',
    'deviceid_prev_day_click_count',
    'deviceid_prev_day_count', 'deviceid_prev_day_ctr',
    'pos_deviceid_prev_day_click_count', 'pos_deviceid_prev_day_count',
    'pos_deviceid_prev_day_ctr', 'newsid_next5_exposure_ts_gap',
    'newsid_next10_exposure_ts_gap', 'pos_deviceid_next10_exposure_ts_gap',
    'pos_newsid_next2_exposure_ts_gap', 'pos_newsid_next3_exposure_ts_gap',
    'pos_newsid_next5_exposure_ts_gap',
    'pos_netmodel_deviceid_next10_exposure_ts_gap', 'newsid_lng_lat_emb_7'
]
# corr_s<0.005
low_corr_feas = [
    'cross_pos_netmodel_count', 'device_vendor', 'deviceid_lng_lat_emb_6',
    'city_level', 'minute',
    'cross_province_device_version_nunique_ratio_province_count',
    'deviceid_lng_lat_prev10_exposure_ts_gap',
    'lng_lat_next10_exposure_ts_gap', 'cross_province_newsid_ent',
    'cross_city_deviceid_ent', 'netmodel_lng_lat_prev5_exposure_ts_gap', 
    'pos_deviceid_lng_lat_prev2_exposure_ts_gap',
    'pos_netmodel_deviceid_lng_lat_prev1_exposure_ts_gap',
    'cross_province_device_version_ent',
    'pos_netmodel_deviceid_next1_exposure_ts_gap',
    'pos_netmodel_lng_lat_next2_exposure_ts_gap',
    'pos_netmodel_deviceid_lng_lat_next2_exposure_ts_gap',
    'lng_lat_newsid_emb_3', 'lng_lat_deviceid_emb_2',
    'netmodel_deviceid_lng_lat_prev5_exposure_ts_gap',
    'cross_city_deviceid_nunique_ratio_city_count'
]

low_importance = ['cross_pos_netmodel_nunique_ratio_pos_count',
                  'cross_lng_lat_deviceid_ent',
                  'cross_netmodel_pos_nunique_ratio_netmodel_count',
                  'cross_pos_newsid_nunique',
                  'cross_netmodel_newsid_nunique',
                  'cross_netmodel_newsid_ent',
                  'cross_lng_lat_deviceid_nunique',
                  'cross_pos_netmodel_nunique',
                  'cross_pos_lng_lat_nunique',
                  'cross_netmodel_pos_nunique',
                  'cross_netmodel_pos_ent',
                  'cross_netmodel_lng_lat_nunique'
                 ]



feas2drop = list(set(feas2drop+low_corr_feas+low_importance))

In [34]:
for i in feas2drop:
    try:
        cate_cols.remove(i)
    except:
        continue
        
        
print(cate_cols)
gc.collect()

['deviceid', 'newsid', 'pos', 'app_version', 'netmodel', 'osversion', 'device_version', 'lng', 'lat', 'lng_lat']


0

In [35]:
for i in feas2drop:
    try:
        #del train_x[i]
        #del val_x[i]
        del train_df[i]
        del test_df[i]
    except:
        continue
print('delete fea finished')
gc.collect()

delete fea finished


0

In [21]:
feas = [i for i in train_df.columns if i not in ['day','id']]

In [22]:
train_df = reduce_mem(train_df)
test_df = reduce_mem(test_df)
gc.collect()

21883.74 Mb, 12574.74 Mb (42.54 %)
7027.91 Mb, 4038.35 Mb (42.54 %)


0

In [ ]:
feas = [i for i in train_df.columns if i not in ['day']]
print('============================== training validate =====================================')
fea_imp_list = []
clf = LGBMClassifier(
    learning_rate=0.03,
    n_estimators=6000,
    num_leaves=255,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=2019,
    metric=None
)
gc.collect()
print('************** training **************')
clf.fit(
    train_x[feas], train_y,
    eval_set=[(val_x[feas], val_y)],
    eval_metric='auc',
    categorical_feature=cate_cols,
    early_stopping_rounds=200,
    verbose=100
)
gc.collect()
print('runtime:', time.time() - t)

print('************** validate predict **************')
best_rounds = clf.best_iteration_
best_auc = clf.best_score_['valid_0']['auc']
val_pred = clf.predict_proba(val_x[feas])[:, 1]
fea_imp_list.append(clf.feature_importances_)
print('runtime:', time.time() - t)

print('============================ training predict ====================================')
clf = LGBMClassifier(
    learning_rate=0.03,
    n_estimators=best_rounds,
    num_leaves=255,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=2019
)
gc.collect()
print('************** training **************')
clf.fit(
    train_df[feas], labels,
    eval_set=[(train_df[feas], labels)],
    categorical_feature=cate_cols,
    verbose=100
)
print('runtime:', time.time() - t)
gc.collect()
print('************** test predict **************')
sub = pd.read_csv('../data/sample.csv')
sub['target'] = clf.predict_proba(test_df[feas])[:, 1]
fea_imp_list.append(clf.feature_importances_)
print('runtime:', time.time() - t)

print('============================ feat importances ====================================')
# 特征重要性可以好好看看
fea_imp_dict = dict(zip(train_df.columns.values, np.mean(fea_imp_list, axis=0)))
fea_imp_item = sorted(fea_imp_dict.items(), key=lambda x: x[1], reverse=True)
for f, imp in fea_imp_item:
    print('{} = {}'.format(f, imp))

In [ ]:
print('============================== threshold search ==================================')
# f1阈值敏感，所以对阈值做一个简单的迭代搜索。
t0 = 0.05
v = 0.002
best_t = t0
best_f1 = 0
for step in range(201):
    curr_t = t0 + step * v
    y = [1 if x >= curr_t else 0 for x in val_pred]
    curr_f1 = f1_score(val_y, y)
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best f1: {}'.format(step, best_t, best_f1))
print('search finish.')

val_pred = [1 if x >= best_t else 0 for x in val_pred]
print('\nbest auc:', best_auc)
print('best f1:', f1_score(val_y, val_pred))
print('validate mean:', np.mean(val_pred))
print('runtime:', time.time() - t)

print('================================= sub save ======================================')
sub.to_csv('sub_prob_{}_{}_{}.csv'.format(best_auc, best_f1, sub['target'].mean()), index=False)
sub['target'] = sub['target'].apply(lambda x: 1 if x >= best_t else 0)
sub.to_csv('sub_{}_{}_{}.csv'.format(best_auc, best_f1, sub['target'].mean()), index=False)
print('runtime:', time.time() - t)
print('finish.')
print('=================================================================================')


# 5flod-lgb

In [1]:
import lightgbm as lgb

In [2]:
param = {  'bagging_freq': 5,
            'bagging_fraction': 0.331,
            'boost_from_average':'false',
            'boost': 'gbdt',
            'feature_fraction': 0.0405,
            'learning_rate': 0.083,
            'max_depth': -1,
            'metric':'auc',
            'min_data_in_leaf': 80,
            'min_sum_hessian_in_leaf': 10.0,
#             'num_leaves': 3,
#             'num_threads': 8,
            'tree_learner': 'serial',
            'objective': 'binary',
            'verbosity': 1
}

In [25]:
feas = [i for i in train_df.columns if i not in ['day','id']]
folds = StratifiedKFold(n_splits=3, shuffle=False, random_state=2319)
oof_lgb =  np.zeros(len(train_df)) 
predictions_lgb =np.zeros(len(test_df))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, labels)):
    print("Fold {}".format(fold_))

    X_train, X_valid = train_df.iloc[trn_idx][feas], train_df.iloc[val_idx][feas]
    y_train, y_valid = labels[trn_idx], labels[val_idx]
     
    trn_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_valid, label=y_valid)   
        
    
    clf = lgb.train(param, 
                    trn_data, 
                    5000, 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval=100, 
                    early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(X_train, num_iteration=clf.best_iteration)
    predictions_lgb += clf.predict(test_df[feas], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(labels, oof_lgb)))


Fold 0


/home/zxh/.local/lib/python3.6/site-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['app_version', 'device_vendor', 'device_version', 'deviceid', 'lat', 'lng', 'lng_lat', 'netmodel', 'newsid', 'osversion', 'pos']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[50]	valid_0's binary_logloss: 0.209725
[100]	valid_0's binary_logloss: 0.164613
[150]	valid_0's binary_logloss: 0.141811
[200]	valid_0's binary_logloss: 0.128746
[250]	valid_0's binary_logloss: 0.120536
[300]	valid_0's binary_logloss: 0.115255
[350]	valid_0's binary_logloss: 0.111579
[400]	valid_0's binary_logloss: 0.10883
[450]	valid_0's binary_logloss: 0.106807
[500]	valid_0's binary_logloss: 0.105241
[550]	valid_0's binary_logloss: 0.103989
[600]	valid_0's binary_logloss: 0.102912
[650]	valid_0's binary_logloss: 0.102036
[700]	valid_0's binary_logloss: 0.101253
[750]	valid_0's binary_logloss: 0.100651
[800]	valid_0's binary_logloss: 0.100162
[850]	valid_0's binary_logloss: 0.0997133
[900]	valid_0's binary_logloss: 0.0993687
[950]	valid_0's binary_logloss: 0.0990755
[1000]	valid_0's binary_logloss: 0.098796
[1050]	valid_0's binary_logloss: 0.0985696
[1100]	valid_0's binary_logloss: 0.0983703
[1150]	valid_0's binary_logloss: 0.0981644
[1200]	valid_0's binary_logloss: 0.0979771
[1250]

KeyboardInterrupt: 

In [ ]:
print('============================ threshold search ============================')
# f1阈值敏感，所以对阈值做一个简单的迭代搜索。
t0 = 0.05
v = 0.002
best_t = t0
best_f1 = 0
for step in range(201):
    curr_t = t0 + step * v
    y = [1 if x >= curr_t else 0 for x in oof2]
    curr_f1 = f1_score(labels, y)
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best f1: {}'.format(step, best_t, best_f1))
print('search finish.')



oof2 = [1 if x >= best_t else 0 for x in oof2]
print('\nbest auc:', roc_auc_score(labels, oof2))
print('best f1:', f1_score(labels , oof2))
print('validate mean:', np.mean(oof2))


print('=================================== sub save =============================')
sub = pd.read_csv('../data/sample.csv')
sub['target'] = predictions2
sub.to_csv('sub_prob_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
sub['target'] = sub['target'].apply(lambda x: 1 if x > best_t else 0)
sub.to_csv('sub_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
print('finish.')
print('==============================================================')


# xgb

In [24]:
import xgboost as xgb

In [3]:
feas = [i for i in train_df.columns if i not in ['day','id']]
params = {'objective': 'binary:logistic',
               'eval_metric': 'auc',
               'max_depth': 14,
               'eta': 0.1,
               'gamma': 6,
               'subsample': 0.8,
               'colsample_bytree': 0.8,
               'min_child_weight': 51,
               'colsample_bylevel': 0.6,
               'lambda': 0.5,
               'alpha': 0.1,
              'scale_pos_weight': 5,
               'silent':0}
    
    
folds = StratifiedKFold(n_splits=3, shuffle=False, random_state=2019)
oof_xgb =  np.zeros(len(train_df)) 
predictions_xgb =np.zeros(len(test_df))

for i, (trn, val) in enumerate(folds.split(train_df[feas].values,labels)):
    print(i+1, "fold.    AUC")
    
    trn_x = train_df.iloc[trn][feas]
    trn_y = labels[trn]
    val_x = train_df.iloc[val][feas]
    val_y = labels[val]

    

    model = xgb.train(params
                      , xgb.DMatrix(trn_x, trn_y)
                      , 200
                      , [(xgb.DMatrix(trn_x, trn_y), 'train'), (xgb.DMatrix(val_x, val_y), 'valid')]
                      , verbose_eval=100
                      , early_stopping_rounds=200
                      ,
                      )

    oof_xgb[val] = model.predict(xgb.DMatrix(val_x), ntree_limit=model.best_ntree_limit)
    predictions_xgb += model.predict(xgb.DMatrix(test_df[feas]), ntree_limit=model.best_ntree_limit) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(labels, oof_xgb)))
cv_auc = roc_auc_score(labels, oof_xgb)


In [27]:
gc.collect()

1503

In [ ]:
print('============================ threshold search ============================')
# f1阈值敏感，所以对阈值做一个简单的迭代搜索。
t0 = 0.05
v = 0.002
best_t = t0
best_f1 = 0
for step in range(201):
    curr_t = t0 + step * v
    y = [1 if x >= curr_t else 0 for x in oof_xgb]
    curr_f1 = f1_score(labels, y)
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best f1: {}'.format(step, best_t, best_f1))
print('search finish.')



oof_xgb_2 = [1 if x >= best_t else 0 for x in oof_xgb]
print('\nbest auc:', roc_auc_score(labels, oof_xgb))
print('best f1:', f1_score(labels , oof_xgb_2))
print('validate mean:', np.mean(oof_xgb_2))


print('=================================== sub save =============================')
sub = pd.read_csv('../data/sample.csv')
sub['target'] = predictions_xgb
sub.to_csv('sub_xgb_prob_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
sub['target'] = sub['target'].apply(lambda x: 1 if x > best_t else 0)
sub.to_csv('sub_xgb_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
print('finish.')
print('==============================================================')


# catboost

In [37]:
from catboost import CatBoostClassifier

In [ ]:
cat_featrreas = cate_cols
feas = [i for i in train_df.columns if i not in ['day']]

folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=2019)
oof_cat =  np.zeros(len(train_df)) 
predictions_cat =np.zeros(len(test_df))
print('model begin')
for i, (trn, val) in enumerate(folds.split(train_df[feas].values,labels)):

    print(i+1, "fold.    AUC")
    
    trn_x = train_df.loc[trn, feas]
    trn_y = labels[trn]
    val_x = train_df.loc[val, feas]
    val_y = labels[val]

    model = CatBoostClassifier(
                            iterations=2500,
                            thread_count=-1,
                            loss_function='Logloss',
                            # depth = 3,
                            # l2_leaf_reg = 29,
                            learning_rate = 0.19,
                            use_best_model=True,
                            task_type="GPU",
#                             devices='0',
                            gpu_ram_part=0.9,
                            eval_metric='AUC',
                            early_stopping_rounds=200,
                            od_type="Iter",
                            verbose=100,
                            cat_features=cat_featrreas,
                            #bagging_temperature = 0.18,
                            #border_count = 136,
                            #random_strength = 0.57,
                            # scale_pos_weight = 0.8
                            )
    model.fit(trn_x, trn_y,
                    eval_set=(val_x, val_y),
                    use_best_model=True,
#                       plot=True
             )
    print('flod ',i+1,'  finished')
    oof_cat[val] = model.predict_proba(val_x)[:,1]
    predictions_cat += model.predict_proba(test_df[feas])[:,1] / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(labels, oof_cat)))
cv_auc = roc_auc_score(labels, oof_cat)

model begin
1 fold.    AUC
0:	learn: 0.9369365	test: 0.9445722	best: 0.9445722 (0)	total: 700ms	remaining: 1h 33m 15s
100:	learn: 0.9740548	test: 0.9773235	best: 0.9773235 (100)	total: 1m 11s	remaining: 1h 33m 14s
200:	learn: 0.9778317	test: 0.9805130	best: 0.9805130 (200)	total: 2m 24s	remaining: 1h 33m 41s
300:	learn: 0.9794760	test: 0.9818072	best: 0.9818072 (300)	total: 3m 35s	remaining: 1h 31m 42s
400:	learn: 0.9804896	test: 0.9826404	best: 0.9826404 (400)	total: 4m 45s	remaining: 1h 30m 13s
500:	learn: 0.9812031	test: 0.9832330	best: 0.9832330 (500)	total: 5m 54s	remaining: 1h 28m 24s
600:	learn: 0.9816883	test: 0.9836264	best: 0.9836264 (600)	total: 7m 4s	remaining: 1h 27m 3s
700:	learn: 0.9820799	test: 0.9839494	best: 0.9839494 (700)	total: 8m 12s	remaining: 1h 25m 32s
800:	learn: 0.9823778	test: 0.9841849	best: 0.9841849 (800)	total: 9m 22s	remaining: 1h 24m 15s
900:	learn: 0.9826504	test: 0.9844061	best: 0.9844061 (900)	total: 10m 32s	remaining: 1h 23m 7s
1000:	learn: 0.98289

In [ ]:
print('============================ threshold search ============================')
# f1阈值敏感，所以对阈值做一个简单的迭代搜索。
t0 = 0.05
v = 0.002
best_t = t0
best_f1 = 0
for step in range(201):
    curr_t = t0 + step * v
    y = [1 if x >= curr_t else 0 for x in oof_cat]
    curr_f1 = f1_score(labels, y)
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best f1: {}'.format(step, best_t, best_f1))
print('search finish.')



oof_cat = [1 if x >= best_t else 0 for x in oof_cat]
print('\nbest auc:', roc_auc_score(labels, oof_cat))
print('best f1:', f1_score(labels , oof_cat))
print('validate mean:', np.mean(oof_cat))


print('=================================== sub save =============================')
sub = pd.read_csv('../data/sample.csv')
sub['target'] = predictions_cat
sub.to_csv('sub_cat_prob_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
sub['target'] = sub['target'].apply(lambda x: 1 if x > best_t else 0)
sub.to_csv('sub_cat_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
print('finish.')
print('==============================================================')


# deepFM

In [30]:
import pandas as pd 
from sklearn.metrics import log_loss, roc_auc_score,f1_score
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler 
from deepctr.models import DeepFM 
from deepctr.inputs import SparseFeat, DenseFeat,get_feature_names
import gc
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
from tqdm import tqdm
import os 

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [31]:
data = train_df.append(test_df,sort = False)
train_len = train_df.shape[0]
data = reduce_mem_usage(data)
del train_df,test_df
gc.collect()
print('read data finished')

Mem. usage decreased to 16613.09 Mb (0.0% reduction)
read data finished


In [37]:
sparse_features = ['deviceid',
 'newsid',
 'pos',
 'app_version',
 'device_vendor',
 'netmodel',
 'osversion',
 'lng',
 'lat',
 'device_version',
 'hour',
 'minute',
 'lng_lat']
dense_features = [i for i in data.columns if i not in sparse_features + ['id','label']]

target = ['label']    


data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )

for feat in tqdm(sparse_features):
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
        gc.collect()
        
for feat in tqdm(dense_features):
        mms = MinMaxScaler(feature_range=(0, 1))
        data[feat] = mms.fit_transform(data[feat].values.reshape(-1,1))
        gc.collect()


fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=8)
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                          for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)


data.reset_index(drop = True,inplace = True)

train = data[:train_len].reset_index(drop=True)
test = data[train_len:].reset_index(drop=True)
del data
gc.collect()

198

In [38]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from keras import backend as K

def metric_precision(y_true,y_pred):    
    TP=tf.reduce_sum(y_true*tf.round(y_pred))
    TN=tf.reduce_sum((1-y_true)*(1-tf.round(y_pred)))
    FP=tf.reduce_sum((1-y_true)*tf.round(y_pred))
    FN=tf.reduce_sum(y_true*(1-tf.round(y_pred)))
    precision=TP/(TP+FP)
    return precision

#召回率评价指标
def metric_recall(y_true,y_pred):  
    TP=tf.reduce_sum(y_true*tf.round(y_pred))
    TN=tf.reduce_sum((1-y_true)*(1-tf.round(y_pred)))
    FP=tf.reduce_sum((1-y_true)*tf.round(y_pred))
    FN=tf.reduce_sum(y_true*(1-tf.round(y_pred)))
    recall=TP/(TP+FN)
    return recall

#F1-score评价指标
def metric_F1score(y_true,y_pred):    
    TP=tf.reduce_sum(y_true*tf.round(y_pred))
    TN=tf.reduce_sum((1-y_true)*(1-tf.round(y_pred)))
    FP=tf.reduce_sum((1-y_true)*tf.round(y_pred))
    FN=tf.reduce_sum(y_true*(1-tf.round(y_pred)))
    precision=TP/(TP+FP)
    recall=TP/(TP+FN)
    F1score=2*precision*recall/(precision+recall)
    return F1score




Using TensorFlow backend.


In [40]:
print('model begin')
class_weights = {0:0.5,1:5}
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary',
               l2_reg_linear=0.1, l2_reg_embedding=0.1,
               l2_reg_dnn=0.1,  seed=1024, dnn_dropout=0.2,
                dnn_activation='relu', dnn_use_bn=True,
              )
model.compile("adam", "binary_crossentropy",
              metrics=['accuracy',metric_F1score], )


folds = StratifiedKFold(n_splits=3, shuffle=False, random_state=44000)
oof_DEEPFM = np.zeros(len(train))
preds_DEEPFM = np.zeros(len(test))


test_model_input = {name:test[name] for name in feature_names}

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values,labels)):
    print("==========flod",fold_,"===============")
    tr = train.iloc[trn_idx]
    tr_y = labels[trn_idx]
    val = train.iloc[val_idx]
    val_y = labels[val_idx]

    tr_model_input = {name:tr[name] for name in feature_names}
    val_model_input = {name:val[name] for name in feature_names}

    history = model.fit(tr_model_input, tr_y,
                        validation_data = (val_model_input,val_y),
                        batch_size=256, epochs=5, verbose=1,class_weight = class_weights)

    oof_DEEPFM[val_idx] = model.predict(val_model_input,batch_size=512)[:,0]
    # pred_train = model.predict(train_model_input, batch_size=512)
    preds_DEEPFM += model.predict(test_model_input,batch_size=512)[:,0]/ folds.n_splits
    # pred_test = model.predict(test_model_input, batch_size=512)
    
print("CV score: {:<8.5f}".format(roc_auc_score(labels, oof_DEEPFM)))

model begin
==========flod 0 ===============
Train on 7584454 samples, validate on 3792227 samples
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 1/5
7584454/7584454 [==============================] - 3835s 506us/sample - loss: 0.5995 - acc: 0.7705 - metric_F1score: 0.4450 - val_loss: 0.8738 - val_acc: 0.3432 - val_metric_F1score: 0.2399
Epoch 2/5
7584454/7584454 [==============================] - 3794s 500us/sample - loss: 0.5519 - acc: 0.7714 - metric_F1score: 0.4460 - val_loss: 0.6828 - val_acc: 0.8931 - val_metric_F1score: nan
Epoch 3/5
7584454/7584454 [==============================] - 3694s 487us/sample - loss: 0.5518 - acc: 0.7717 - metric_F1score: 0.4461 - val_loss: 0.5982 - val_acc: 0.8935 - val_metric_F1score: nan
Epoch 4/5
7584454/7584454 [==============================] - 3589s 473us/sample - loss: 0.5513 - acc: 0.7720 - metric_F1score: 0.4466 - val_loss: 0.7903 - val_acc: 0.5854 - val_metric_F1score: 0.2899
Epoch 5/5
7584454/7584454 [==============================] - 3819s 504us/sample - loss: 0.5515 - acc: 0.7715 - metric_F1score: 0.4460 - val_loss: 0.6816 - val_acc: 0.8922 - val_metric_F1score: nan
=========

In [41]:
print('============================ threshold search ============================')
# f1阈值敏感，所以对阈值做一个简单的迭代搜索。
t0 = 0.05
v = 0.002
best_t = t0
best_f1 = 0
for step in range(201):
    curr_t = t0 + step * v
    y = [1 if x >= curr_t else 0 for x in oof_DEEPFM]
    curr_f1 = f1_score(labels, y)
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best f1: {}'.format(step, best_t, best_f1))
print('search finish.')



oof_DEEPFM_1 = [1 if x >= best_t else 0 for x in oof_DEEPFM]
print('\nbest auc:', roc_auc_score(labels, oof_DEEPFM))
print('best f1:', f1_score(labels , oof_DEEPFM_1))
print('validate mean:', np.mean(oof_DEEPFM))


print('=================================== sub save =============================')
sub = pd.read_csv('../data/sample.csv')
sub['target'] = preds_DEEPFM
sub.to_csv('sub_deepfm_prob_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
sub['target'] = sub['target'].apply(lambda x: 1 if x > best_t else 0)
sub.to_csv('sub_deepfm_5flod_base_{}_{}.csv'.format(best_f1, sub['target'].mean()), index=False)
print('finish.')
print('==============================================================')


============================ threshold search ============================
step: 0   best threshold: 0.05   best f1: 0.1925565143784914
step: 105   best threshold: 0.26   best f1: 0.19255652967447465
step: 107   best threshold: 0.264   best f1: 0.1925565449704604
step: 108   best threshold: 0.266   best f1: 0.19255657556243913
step: 109   best threshold: 0.268   best f1: 0.19255682029861906
step: 110   best threshold: 0.27   best f1: 0.1925571415157994
step: 111   best threshold: 0.272   best f1: 0.19255787573052147
step: 112   best threshold: 0.274   best f1: 0.19255888528490667
step: 113   best threshold: 0.276   best f1: 0.19256050671258818
step: 114   best threshold: 0.278   best f1: 0.19256382613555958
step: 115   best threshold: 0.28   best f1: 0.1925694097549257
step: 116   best threshold: 0.28200000000000003   best f1: 0.19257650830291137
step: 117   best threshold: 0.28400000000000003   best f1: 0.1925874113317829
step: 118   best threshold: 0.28600000000000003   best f1: 0.19

In [42]:
train_result = pd.DataFrame({'real_target':labels,'pred_target':oof_DEEPFM})

In [44]:
train_result.to_pickle('train_deepfm_rseult.pickle')

# NN 模型

In [3]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
data = pd.concat([train_df,test_df],axis = 0,sort = False)

In [4]:
app_data = pd.read_pickle('../pickle/device_new_app.pickle')
tag_data = pd.read_pickle('../pickle/tag_weight_new_data.pickle')

In [5]:
data.shape

(15030273, 15)

In [6]:
data_label = data['deviceid']
del train_df
del test_df
del data
gc.collect()

0

In [7]:
data_label = pd.DataFrame(data_label)

In [10]:
data_label = data_label.merge(app_data,how = 'left',on = 'deviceid')
data_label = data_label.merge(tag_data,how = 'left',on = 'deviceid')

In [12]:
del data_label['app_len']
del data_label['all_tag_weight']

data_label.head()

,deviceid,applist,all_tag_word
0,8b2d7f2aed47ab32e9c6ae4f5ae00147,[app_133],[未知]
1,8b2d7f2aed47ab32e9c6ae4f5ae00147,[app_133],[未知]
2,832aaa33cdf4a0938ba2c795eb3ffefd,"[app_1, app_2, app_3, app_4, app_5, app_6, app...","[乔振宇, 婚姻, 离婚, 杨紫, 约会, 出轨, 夫妻, 浪漫, 王子文, 合照, 婆媳,..."
3,832aaa33cdf4a0938ba2c795eb3ffefd,"[app_1, app_2, app_3, app_4, app_5, app_6, app...","[乔振宇, 婚姻, 离婚, 杨紫, 约会, 出轨, 夫妻, 浪漫, 王子文, 合照, 婆媳,..."
4,67dd9dac18cce1a6d79e8f20eefd98ab,"[app_84, app_85, app_4, app_5, app_86, app_87,...","[片段, 社会热点, 美食, --其他, 花絮片段, 陆军, 坦克, 机枪, 家庭, 武器,..."


In [2]:
### Tokenizer 序列化文本
def set_tokenizer(docs, split_char=' ', max_len=100):
    '''
    输入
    docs:文本列表
    split_char:按什么字符切割
    max_len:截取的最大长度
    
    输出
    X:序列化后的数据
    word_index:文本和数字对应的索引
    '''
    tokenizer = Tokenizer(lower=False, char_level=False, split=split_char)
    tokenizer.fit_on_texts(docs)
    X = tokenizer.texts_to_sequences(docs)
    maxlen = max_len
    X = pad_sequences(X, maxlen=maxlen, value=0)
    word_index=tokenizer.word_index
    return X, word_index

### 做embedding 这里采用word2vec 可以换成其他例如（glove词向量）
def trian_save_word2vec(docs, embed_size=300, save_name='w2v.txt', split_char=' '):
    '''
    输入
    docs:输入的文本列表
    embed_size:embed长度
    save_name:保存的word2vec位置
    
    输出
    w2v:返回的模型
    '''
    input_docs = []
    for i in docs:
        input_docs.append(i.split(split_char))
    logging.basicConfig(
    format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    w2v = Word2Vec(input_docs, size=embed_size, sg=1, window=8, seed=1017, workers=24, min_count=1, iter=10)
    w2v.wv.save_word2vec_format(save_name)
    print("w2v model done")
    return w2v

# 得到embedding矩阵
def get_embedding_matrix(word_index, embed_size=300, Emed_path="w2v_300.txt"):
    embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(
        Emed_path, binary=False)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector    
    print("null cnt",count)
    return embedding_matrix

In [14]:
def get_list_data(row):
    review = ''
    for i in row:
        review += i + ' '
    return review

text_1_list = list(data_label['applist'].apply(lambda row:get_list_data(row)))
text_2_list = list(data_label['all_tag_word'].apply(lambda row:get_list_data(row)))

In [ ]:
x1, index_1 = set_tokenizer(text_1_list, split_char=' ', max_len=160)
x2, index_2 = set_tokenizer(text_2_list, split_char=' ', max_len=160)

In [ ]:
# 值得提醒的是这个保存方法是采用w2v.wv.save_word2vec_format
# 因此你如果载入自己训练模型的时候，需要载入后再按照这个函数来保存再在emed_path中输入
trian_save_word2vec(text_1_list, save_name='w2v_model/cate_w2v_300.txt', split_char=' ')
gc.collect()
trian_save_word2vec(text_1_list, save_name='w2v_model/w2v_300.txt', split_char=' ')
gc.collect()
# 得到emb矩阵
emb1 = get_embedding_matrix(index_1, Emed_path='w2v_model/cate_w2v_300.txt')
emb2 = get_embedding_matrix(index_2, Emed_path='w2v_model/w2v_300.txt')
gc.collect()

In [ ]:
df = df.fillna(-1)
# 处理类别特征
for col in cate_fea:
    df[col] = df[col].map(df[col].value_counts().rank()/len(df[col].unique()))
# 处理数值特征
for i in num_fea
    ss=StandardScaler()
    df[col] = ss.fit_transform(df[col])
num_feature_input = len(cate_fea + num_fea)

In [ ]:
# 区分开train和valid,test
# 这里是假设三输入
train_data = data[data['age_group']!=-1]
train_input_1 = x1[:len(train_data)]
test_input_1 = x1[len(train_data):]
train_input_2 = x2[:len(train_data)]
test_input_2 = x2[len(train_data):]
train_input_3 = df[:len(train_data)][]
test_input_3 = df[len(train_data):][]
label = labels

In [3]:
# 得到emb矩阵
index_1 = np.load('NN_model/index_1.npy',allow_pickle = 'True')
index_2 = np.load('NN_model/index_2.npy',allow_pickle = 'True')
emb1 = get_embedding_matrix(index_1.tolist(), Emed_path='NN_model/app_w2v_300.txt')
emb2 = get_embedding_matrix(index_2.tolist(), Emed_path='NN_model/tag_w2v_300.txt')
gc.collect()

/home/zsy/anaconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



null cnt 25731



null cnt 7


0

In [12]:
from keras.initializers import *

def model_conv(emb1, emb3, num_feature_input):
    '''
    注意这个inputs
    seq1、seq2分别是两个输入
    hin是feature层输入
    是否做emb可选可不选，
    这个就是我们之前训练已经得到的用于embedding的（embedding_matrix1， embedding_matrix2）
    '''
    K.clear_session()

    emb_layer_1 = Embedding(
        input_dim=emb1.shape[0],
        output_dim=emb1.shape[1],
        weights=[emb1],
        input_length=90,
        trainable=False
    )
    
    emb_layer_3 = Embedding(
        input_dim=emb3.shape[0],
        output_dim=emb3.shape[1],
        weights=[emb3],
        input_length=600,
        trainable=False
    )
    
    
    seq1 = Input(shape=(90,))
    seq3 = Input(shape=(600,))    
    
    x1 = emb_layer_1(seq1)
    x3 = emb_layer_3(seq3)
    
    sdrop=SpatialDropout1D(rate=0.2)

    x1 = sdrop(x1)
    x3 = sdrop(x3)
    
    x = Dropout(0.2)(Bidirectional(CuDNNLSTM(200, return_sequences=True))(x1))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    merged_1_avg = Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,))(semantic)
    
    x = Dropout(0.2)(Bidirectional(CuDNNLSTM(200, return_sequences=True))(x3))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_3 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    merged_3_avg = Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,))(semantic)
    
    hin = Input(shape=(num_feature_input, ))
    htime = Dense(16, activation='relu')(hin)
    
    x = concatenate([merged_1, merged_3, merged_1_avg, merged_3_avg, htime])
    
    x = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(1000)(x))))
    x = Activation(activation="relu")(BatchNormalization()(Dense(500)(x)))
    x = Activation(activation="relu")(BatchNormalization()(Dense(120)(x)))
    pred = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[seq1, seq3, hin], outputs=pred)
    from keras.utils import multi_gpu_model
#     model = multi_gpu_model(model, 2)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=0.001),metrics=["accuracy"])
    return model
gc.collect()

7668

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=1017, shuffle=True)
oof_pred = np.zeros(len(train_input_3))
sub = np.zeros(len(test_input_3))
score = []
count = 0
if not os.path.exists("nn_model"):
    os.mkdir("nn_model")

for i, (train_index, test_index) in enumerate(skf.split(train_input_3, labels)):
    print("FOLD | ", count+1)
    print("###"*35)
    gc.collect()
    filepath = "nn_model/nn_v1_%d.h5" % count
    checkpoint = ModelCheckpoint(
        filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
    reduce_lr = ReduceLROnPlateau(
        monitor='val_acc', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
    earlystopping = EarlyStopping(
        monitor='val_acc', min_delta=0.0001, patience=5, verbose=1, mode='max')
    callbacks = [checkpoint, reduce_lr, earlystopping]
    model_ctr = model_conv(emb1, emb2, num_feature_input)
    if count==0:model_age.summary()
    x1_tr, x1_va = np.array(train_input_1)[train_index], np.array(train_input_1)[test_index]    
    x2_tr, x2_va = np.array(train_input_2)[train_index], np.array(train_input_2)[test_index]
    x3_tr, x3_va = np.array(train_input_3)[train_index], np.array(train_input_3)[test_index]
    y_tr, y_va = labels[train_index], labels[test_index]
    
    hist = model_ctr.fit([x1_tr, x2_tr, x3_tr],
                         y_tr, batch_size=4096, epochs=50, 
                         validation_data=([x1_va, x2_va, x3_va], y_va),
                         callbacks=callbacks, verbose=1, shuffle=True)

    model_ctr.load_weights(filepath)
    oof_pred[test_index] = model_ctr.predict([x1_va, x2_va, x3_va],batch_size=2048,verbose=1)
    sub += model_ctr.predict([test_input_1, test_input_2, test_input_3],batch_size=2048,verbose=1)/skf.n_splits
    score.append(np.max(hist.history['val_acc']))
    count += 1
print('acc:', np.mean(score))